# 1- Project Description

## What the System Does
This is a **Budget Analysis System** that helps users manage their financial transactions. Users can:
- Add income and expense transactions
- View all transactions
- Calculate total income, expenses, and balance
- Analyze spending by category
- Detect suspicious/fraudulent activity

## How Transactions are Managed
- Each transaction is stored as a `Transaction` object with date, amount, category, type, and description
- All transactions are stored in a `BudgetTracker` class using a list
- Users interact with the system through a menu-driven interface

## How Suspicious Activity is Detected
- Large expenses exceeding a threshold are flagged
- Multiple expenses in the same category are considered suspicious
- The system uses dictionaries with tuple keys to track patterns

## Python Concepts Used
- **Classes**: Transaction, BudgetTracker
- **Functions**: Validation, creation, fraud detection
- **Loops**: for, while
- **Conditions**: if/elif/else
- **Lists**: Store transactions
- **Dictionaries**: Spending by category, fraud detection
- **Sets**: Unique categories
- **Tuples**: Transaction data, dictionary keys
- **List Comprehensions**: Filter and transform data
- **Multiple arguments & return values**
- **Nested logic**: Fraud detection

In [33]:
# 2️ - Constants & Data Rules (Code)

Transaction_Categories = {"Rent", "Salaries", "Electricity", "Material", "Food", "Transport", "Entertainment", "Health", "Other"}

Accepted_Types = ("Income", "Expense")

LARGE_EXPENSE_THRESHOLD = 5000
SUSPICIOUS_THRESHOLD = 3000

In [34]:
# 3 - Core Data Model (Classes)

class Transaction:
    def __init__(self, Date, Amount, Category, Type, Description):
        self.Date = Date
        self.Amount = Amount
        self.Category = Category
        self.Type = Type
        self.Description = Description

    def info(self):
        return (self.Date, self.Amount, self.Category, self.Type, self.Description)


class BudgetTracker:
    def __init__(self, Budget=0):
        self.budget = Budget
        self.Transactions = []

    def add_Transaction(self, transaction):
        self.Transactions.append(transaction)

    def calculate_Totals(self):
        expenses = 0
        income = 0
        for t in self.Transactions:
            if t.Type == "Income":
                income = income + t.Amount
            else:
                expenses = expenses + t.Amount
        return expenses, income
    
    def current_balance(self):
        expenses, income = self.calculate_Totals()
        return income - expenses
    
    def spending_by_category(self):
        spendings = {}
        for t in self.Transactions:
            if t.Type == "Expense":
                if t.Category in spendings:
                    spendings[t.Category] = spendings[t.Category] + t.Amount
                else:
                    spendings[t.Category] = t.Amount
        return spendings

    def unique_categories_used(self):
        return {t.Category for t in self.Transactions}
    
    def summary(self):
        total_income = 0
        total_expenses = 0
        by_category = {}
        
        for t in self.Transactions:
            if t.Type == "Income":
                total_income = total_income + t.Amount
            else:
                total_expenses = total_expenses + t.Amount
                if t.Category in by_category:
                    by_category[t.Category] = by_category[t.Category] + t.Amount
                else:
                    by_category[t.Category] = t.Amount
        
        balance = total_income - total_expenses
        return total_income, total_expenses, balance, by_category
    
    def get_all_transactions(self):
        return self.Transactions

In [35]:
# 4️ - Validation Functions

def validate_Amount(amount):
    if amount > 0:
        return True
    else:
        return False

def validate_Category(category):
    if category in Transaction_Categories:
        return True
    else:
        return False

def validate_Type(trans_type):
    if trans_type in Accepted_Types:
        return True
    else:
        return False

In [36]:
# 5️ - Transaction Creation Logic

def create_Transaction(Date, Amount, Category, Type, Description):
    if validate_Amount(Amount) == False:
        print("Error: Amount must be positive!")
        return None, False
    
    if validate_Category(Category) == False:
        print(f"Error: Invalid category!")
        return None, False
    
    if validate_Type(Type) == False:
        print("Error: Type must be 'Income' or 'Expense'!")
        return None, False
    
    transaction = Transaction(Date, Amount, Category, Type, Description)
    
    is_large = False
    if Type == "Expense" and Amount > LARGE_EXPENSE_THRESHOLD:
        is_large = True
        print(f"Warning: This is a large expense!")
    
    return transaction, is_large

In [37]:
# 7️ - List Comprehension Tasks

def get_high_value_expenses(tracker, threshold):
    high_expenses = [t for t in tracker.Transactions if t.Type == "Expense" and t.Amount > threshold]
    return high_expenses

def transactions_to_tuples(tracker):
    tuples_list = [t.info() for t in tracker.Transactions]
    return tuples_list

def get_expenses_only(tracker):
    expenses = [t for t in tracker.Transactions if t.Type == "Expense"]
    return expenses

def get_income_only(tracker):
    income = [t for t in tracker.Transactions if t.Type == "Income"]
    return income

In [38]:
# 8️ - Fraud Detection System

def fraud_Detection(tracker):
    suspicious_large = []
    suspicious_duplicates = {}
    expense_tracker = {}
    
    for t in tracker.Transactions:
        if t.Type == "Expense":
            if t.Amount > SUSPICIOUS_THRESHOLD:
                suspicious_large.append(t)
            
            key = (t.Category, t.Type)
            
            if key in expense_tracker:
                expense_tracker[key].append(t)
            else:
                expense_tracker[key] = [t]
    
    for key in expense_tracker:
        transactions = expense_tracker[key]
        if len(transactions) > 1:
            category, trans_type = key
            suspicious_duplicates[key] = transactions
    
    total_suspicious_count = len(suspicious_large) + len(suspicious_duplicates)
    
    return suspicious_large, suspicious_duplicates, total_suspicious_count


def display_fraud_report(tracker):
    suspicious_large, suspicious_duplicates, total_count = fraud_Detection(tracker)
    
    print("\n" + "="*50)
    print("FRAUD DETECTION REPORT")
    print("="*50)
    
    print(f"\nLarge Expenses (>${SUSPICIOUS_THRESHOLD}):")
    if len(suspicious_large) > 0:
        for t in suspicious_large:
            print(f"   - {t.info()}")
    else:
        print("   No large suspicious expenses found.")
    
    print(f"\nMultiple Expenses in Same Category:")
    if len(suspicious_duplicates) > 0:
        for key in suspicious_duplicates:
            transactions = suspicious_duplicates[key]
            category, trans_type = key
            print(f"   Category: {category}")
            for t in transactions:
                print(f"      - ${t.Amount}: {t.Description}")
    else:
        print("   No duplicate suspicious expenses found.")
    
    print(f"\nTotal Suspicious Activities: {total_count}")
    print("="*50)

In [39]:
# 9️ - Interactive Menu System

def display_menu():
    print("\n" + "="*40)
    print("BUDGET TRACKER MENU")
    print("="*40)
    print("1 - Add Income")
    print("2 - Add Expense")
    print("3 - View All Transactions")
    print("4 - View Balance")
    print("5 - View Spending by Category")
    print("6 - View Unique Categories")
    print("7 - Run Fraud Detection")
    print("8 - View High Value Expenses")
    print("9 - Exit")
    print("="*40)


def add_income(tracker):
    print("\n--- Add Income ---")
    date = input("Enter date: ")
    amount = float(input("Enter amount: "))
    
    print("Categories:", Transaction_Categories)
    category = input("Enter category: ")
    description = input("Enter description: ")
    
    transaction, is_large = create_Transaction(date, amount, category, "Income", description)
    
    if transaction != None:
        tracker.add_Transaction(transaction)
        print("Income added successfully!")


def add_expense(tracker):
    print("\n--- Add Expense ---")
    date = input("Enter date: ")
    amount = float(input("Enter amount: "))
    
    print("Categories:", Transaction_Categories)
    category = input("Enter category: ")
    description = input("Enter description: ")
    
    transaction, is_large = create_Transaction(date, amount, category, "Expense", description)
    
    if transaction != None:
        tracker.add_Transaction(transaction)
        print("Expense added successfully!")


def view_transactions(tracker):
    print("\n--- All Transactions ---")
    transactions = tracker.get_all_transactions()
    
    if len(transactions) == 0:
        print("No transactions found.")
    else:
        count = 1
        for t in transactions:
            print(f"{count}. {t.info()}")
            count = count + 1


def view_balance(tracker):
    print("\n--- Balance Summary ---")
    expenses, income = tracker.calculate_Totals()
    balance = tracker.current_balance()
    
    print(f"Total Income:   ${income}")
    print(f"Total Expenses: ${expenses}")
    print(f"Current Balance: ${balance}")


def view_spending_by_category(tracker):
    print("\n--- Spending by Category ---")
    spending = tracker.spending_by_category()
    
    if len(spending) == 0:
        print("No expenses recorded.")
    else:
        for category in spending:
            amount = spending[category]
            print(f"  {category}: ${amount}")


def view_unique_categories(tracker):
    print("\n--- Unique Categories Used ---")
    categories = tracker.unique_categories_used()
    
    if len(categories) == 0:
        print("No categories used yet.")
    else:
        for cat in categories:
            print(f"  - {cat}")


def view_high_value_expenses(tracker):
    print("\n--- High Value Expenses ---")
    threshold = float(input("Enter threshold: "))
    
    high_expenses = get_high_value_expenses(tracker, threshold)
    
    if len(high_expenses) == 0:
        print(f"No expenses above ${threshold}")
    else:
        print(f"Expenses above ${threshold}:")
        for t in high_expenses:
            print(f"  - {t.info()}")


def run_menu():
    tracker = BudgetTracker(10000)
    
    print("\n" + "="*50)
    print("Welcome to the Budget Tracker System!")
    print("="*50)
    
    while True:
        display_menu()
        choice = input("Enter your choice (1-9): ")
        
        if choice == "1":
            add_income(tracker)
        elif choice == "2":
            add_expense(tracker)
        elif choice == "3":
            view_transactions(tracker)
        elif choice == "4":
            view_balance(tracker)
        elif choice == "5":
            view_spending_by_category(tracker)
        elif choice == "6":
            view_unique_categories(tracker)
        elif choice == "7":
            display_fraud_report(tracker)
        elif choice == "8":
            view_high_value_expenses(tracker)
        elif choice == "9":
            print("\nThank you for using Budget Tracker. Goodbye!")
            break
        else:
            print("Invalid choice! Please enter 1-9.")
    
    return tracker

# 10 - Testing & Reflection

## Testing Strategy
- 10 transactions (mix of income and expenses)
- Include transactions that trigger suspicious activity detection
- Test all menu options
- Demonstrate list comprehensions and all Python concepts

In [40]:
# 10 - Testing: Add Sample Transactions

# Create a BudgetTracker for testing
test_tracker = BudgetTracker(10000)

# Sample transactions data (at least 10 transactions)
sample_data = [
    ("01-12-2025", 5000, "Salaries", "Income", "Monthly salary"),
    ("02-12-2025", 1200, "Rent", "Expense", "Office rent"),
    ("03-12-2025", 350, "Electricity", "Expense", "Electric bill"),
    ("05-12-2025", 200, "Food", "Expense", "Team lunch"),
    ("05-12-2025", 150, "Food", "Expense", "Office snacks"),
    ("07-12-2025", 800, "Transport", "Expense", "Fuel costs"),
    ("10-12-2025", 2000, "Material", "Expense", "Office supplies"),
    ("12-12-2025", 3500, "Material", "Expense", "New equipment"),
    ("15-12-2025", 500, "Health", "Expense", "Medical insurance"),
    ("15-12-2025", 300, "Health", "Expense", "First aid supplies"),
    ("20-12-2025", 1500, "Entertainment", "Expense", "Team building"),
    ("25-12-2025", 10000, "Salaries", "Income", "Bonus payment"),
]

# Add all sample transactions
print("Adding sample transactions...")
print("-" * 50)

for data in sample_data:
    date, amount, category, trans_type, description = data
    transaction, is_large = create_Transaction(date, amount, category, trans_type, description)
    
    if transaction != None:
        test_tracker.add_Transaction(transaction)
        print(f"Added: {transaction.info()}")

print("-" * 50)
print(f"Total transactions added: {len(test_tracker.Transactions)}")

Adding sample transactions...
--------------------------------------------------
Added: ('01-12-2025', 5000, 'Salaries', 'Income', 'Monthly salary')
Added: ('02-12-2025', 1200, 'Rent', 'Expense', 'Office rent')
Added: ('03-12-2025', 350, 'Electricity', 'Expense', 'Electric bill')
Added: ('05-12-2025', 200, 'Food', 'Expense', 'Team lunch')
Added: ('05-12-2025', 150, 'Food', 'Expense', 'Office snacks')
Added: ('07-12-2025', 800, 'Transport', 'Expense', 'Fuel costs')
Added: ('10-12-2025', 2000, 'Material', 'Expense', 'Office supplies')
Added: ('12-12-2025', 3500, 'Material', 'Expense', 'New equipment')
Added: ('15-12-2025', 500, 'Health', 'Expense', 'Medical insurance')
Added: ('15-12-2025', 300, 'Health', 'Expense', 'First aid supplies')
Added: ('20-12-2025', 1500, 'Entertainment', 'Expense', 'Team building')
Added: ('25-12-2025', 10000, 'Salaries', 'Income', 'Bonus payment')
--------------------------------------------------
Total transactions added: 12


In [41]:
# Test: View All Transactions
print("\n📋 ALL TRANSACTIONS:")
print("="*60)
view_transactions(test_tracker)


📋 ALL TRANSACTIONS:

--- All Transactions ---
1. ('01-12-2025', 5000, 'Salaries', 'Income', 'Monthly salary')
2. ('02-12-2025', 1200, 'Rent', 'Expense', 'Office rent')
3. ('03-12-2025', 350, 'Electricity', 'Expense', 'Electric bill')
4. ('05-12-2025', 200, 'Food', 'Expense', 'Team lunch')
5. ('05-12-2025', 150, 'Food', 'Expense', 'Office snacks')
6. ('07-12-2025', 800, 'Transport', 'Expense', 'Fuel costs')
7. ('10-12-2025', 2000, 'Material', 'Expense', 'Office supplies')
8. ('12-12-2025', 3500, 'Material', 'Expense', 'New equipment')
9. ('15-12-2025', 500, 'Health', 'Expense', 'Medical insurance')
10. ('15-12-2025', 300, 'Health', 'Expense', 'First aid supplies')
11. ('20-12-2025', 1500, 'Entertainment', 'Expense', 'Team building')
12. ('25-12-2025', 10000, 'Salaries', 'Income', 'Bonus payment')


In [42]:
# Test: View Balance
print("\nBALANCE SUMMARY:")
print("="*60)
view_balance(test_tracker)


BALANCE SUMMARY:

--- Balance Summary ---
Total Income:   $15000
Total Expenses: $10500
Current Balance: $4500


In [43]:
# Test: Spending by Category
print("\nSPENDING BY CATEGORY:")
print("="*60)
view_spending_by_category(test_tracker)


SPENDING BY CATEGORY:

--- Spending by Category ---
  Rent: $1200
  Electricity: $350
  Food: $350
  Transport: $800
  Material: $5500
  Health: $800
  Entertainment: $1500


In [44]:
# Test: Unique Categories (Set Comprehension)
print("\nUNIQUE CATEGORIES USED:")
print("="*60)
view_unique_categories(test_tracker)


UNIQUE CATEGORIES USED:

--- Unique Categories Used ---
  - Rent
  - Salaries
  - Entertainment
  - Health
  - Material
  - Transport
  - Food
  - Electricity


In [45]:
# Test: Fraud Detection (Triggers suspicious activity)
print("\nFRAUD DETECTION:")
print("="*60)
display_fraud_report(test_tracker)


FRAUD DETECTION:

FRAUD DETECTION REPORT

Large Expenses (>$3000):
   - ('12-12-2025', 3500, 'Material', 'Expense', 'New equipment')

Multiple Expenses in Same Category:
   Category: Food
      - $200: Team lunch
      - $150: Office snacks
   Category: Material
      - $2000: Office supplies
      - $3500: New equipment
   Category: Health
      - $500: Medical insurance
      - $300: First aid supplies

Total Suspicious Activities: 4


In [46]:
# Test: List Comprehensions
print("\nLIST COMPREHENSION TESTS:")
print("="*60)

# High value expenses (list comprehension)
print("\n1. High-Value Expenses (>$1000):")
high_expenses = get_high_value_expenses(test_tracker, 1000)
for t in high_expenses:
    print(f"   - {t.info()}")

# All transactions as tuples (list comprehension)
print("\n2. All Transactions as Tuples:")
tuples = transactions_to_tuples(test_tracker)
count = 1
for tup in tuples:
    if count <= 3:
        print(f"   {count}. {tup}")
    count = count + 1
print(f"   ... and {len(tuples)-3} more")

# Expenses only (list comprehension)
print(f"\n3. Total Expense Transactions: {len(get_expenses_only(test_tracker))}")
print(f"4. Total Income Transactions: {len(get_income_only(test_tracker))}")


LIST COMPREHENSION TESTS:

1. High-Value Expenses (>$1000):
   - ('02-12-2025', 1200, 'Rent', 'Expense', 'Office rent')
   - ('10-12-2025', 2000, 'Material', 'Expense', 'Office supplies')
   - ('12-12-2025', 3500, 'Material', 'Expense', 'New equipment')
   - ('20-12-2025', 1500, 'Entertainment', 'Expense', 'Team building')

2. All Transactions as Tuples:
   1. ('01-12-2025', 5000, 'Salaries', 'Income', 'Monthly salary')
   2. ('02-12-2025', 1200, 'Rent', 'Expense', 'Office rent')
   3. ('03-12-2025', 350, 'Electricity', 'Expense', 'Electric bill')
   ... and 9 more

3. Total Expense Transactions: 10
4. Total Income Transactions: 2


In [47]:
# Test: Summary (Uses loops, conditions, tuple unpacking)
print("\n FULL SUMMARY:")
print("="*60)
income, expenses, balance, by_cat = test_tracker.summary()
print(f"Total Income:   ${income}")
print(f"Total Expenses: ${expenses}")
print(f"Balance:        ${balance}")
print("\nSpending By Category:")
for cat in by_cat:
    amt = by_cat[cat]
    print(f"  {cat}: ${amt}")


 FULL SUMMARY:
Total Income:   $15000
Total Expenses: $10500
Balance:        $4500

Spending By Category:
  Rent: $1200
  Electricity: $350
  Food: $350
  Transport: $800
  Material: $5500
  Health: $800
  Entertainment: $1500


In [48]:
#run_menu()